In [155]:
import csv
import random
from collections import defaultdict
from copy import deepcopy
from typing import List

In [2]:
out = csv.reader(open('Eldrow Wordlist - Sheet1.csv', 'r'))
wordList = [line[0] for line in out]

In [3]:
import matplotlib.pyplot as plt

In [4]:
# frequencies of letters at different positions
freq = {i: defaultdict(int) for i in range(5)}
for i in range(5):
    for word in wordList:
        freq[i][word[i]] += 1

In [172]:
# Game Engine
class WordleHardMode: 
    """
    ## four types of colour ##
    0. no colour (initial phase)
    1. grey - non-existing alphabet
    2. yellow - correct alphabet but incorrect location
    3. green - correct alphabet with correct location
    
    """
    
    
    def __init__(self, wordListFile = 'Eldrow Wordlist - Sheet1.csv',
                verbose = False):
        # game 
        self.targetWord = None
        self.colours = ["W"] * 5
        self.loadWordList(wordListFile)
        self.guesses = []
        self.finished = False
        
        # helper variables
        self.greens = [''] * 5
        self.yellows = [[],[],[],[],[]]
        self.greys = []
        
        # game settings
        self.verbose = verbose
    
    # setup
    def loadWordList(self, f: str) -> None:
        out = csv.reader(open(f, 'r'))
        self.WORDLIST = [line[0] for line in out]
        self.validWords = self.WORDLIST
        
    def randomTarget(self) -> None:
        self.targetWord = random.choice(self.WORDLIST)
        
    def setTarget(self, targetWord: str):
        assert targetWord in self.WORDLIST, "Word not in word list"
        self.targetWord = targetWord
        
    # game
    def step(self, guessWord: str) -> WordleHardMode:
        gs = self._guess(guessWord)
        if not gs.finished:
            gs._getValidWords()
        return gs
        
    def getLegalMoves(self) -> List[str]:
        return self.validWords
    
    # helper function
    def _guess(self, guessWord: str) -> WordleHardMode:
        if guessWord not in self.validWords:
#             print("Invalid word. Try again!")
            print(f"{guessWord} is not in {self.validWords}")
            return
        nextGameState = deepcopy(self)
        nextGameState.guesses.append(guessWord)
        if (guessWord == nextGameState.targetWord):
            if self.verbose:
                print("Game finished!")
            nextGameState.finished = True
            for i in range(5):
                nextGameState.colours[i] = "G"
            return nextGameState
        
        for i in range(5):
            # marked for green firsts
            if guessWord[i] == nextGameState.targetWord[i]:
                nextGameState.colours[i] = "G"
                nextGameState.greens[i] = guessWord[i]
                continue
            # marked yellow or grey
            nextGameState.colours[i] = "X"
            if guessWord[i] in self.targetWord and guessWord[i] != nextGameState.targetWord[i]:
                for j, targetChar in enumerate(self.targetWord):
                    if i == j:
                        continue
                    if guessWord[i] == targetChar and \
                        (self.colours[j] != "G" or guessWord[j] == nextGameState.targetWord[j]):
                        nextGameState.colours[i] = "Y"
                        break
            if nextGameState.colours[i] == "Y":
                nextGameState.yellows[i].append(guessWord[i])
            else:
                nextGameState.greys.append(guessWord[i])
        if nextGameState.verbose:
            print(guessWord)
            print("".join(nextGameState.colours))
        return nextGameState
        
    
    def _getValidWords(self) -> None:
        validWords = []
        for word in self.validWords:
            valid = True
            for idx, char in enumerate(word):
                # condition: green letter must be used again
                if self.greens[idx] != '' and char != self.greens[idx]:
                    valid = False
                    continue
                # condition: yellow cannot be played in the same position
                if char in self.yellows[idx]:
                    valid = False
                    continue
                # condition: grey cannot be played in any position
                if char in self.greys:
                    valid = False
                    continue
            # condition: check if all yellows are played
            for posV in self.yellows:
                for charYellow in posV:
                    if charYellow not in word:
                        valid = False
                        continue
            if valid:
                validWords.append(word)
        self.validWords = validWords
        

    def debug(self):
        print(self.greens)
        print(self.yellows)
        print(self.greys)
        
    def __str__(self):
        if len(self.guesses ) == 0:
            return
        return f"Round {len(self.guesses)}:\n{self.guesses[-1]}\n{''.join(self.colours)}"

In [173]:
gameState = WordleHardMode()
gameState.setTarget("APPLE")

In [174]:
# # test
# nextGameState = gameState.guess("EXIST")
# print(nextGameState)
# print(len(nextGameState.getValidWords()))

In [175]:
def getMaxWord(gameState):
    maxWord = ''
    maxOptions = -1
    for word in gameState.validWords:
#         print(word)
        nextGameState = gameState.step(word)
        if nextGameState.finished:
            continue
        optionsCount = len(nextGameState.validWords)
        if optionsCount > maxOptions:
            maxOptions = optionsCount
            maxWord = word
    return maxWord, maxOptions

In [176]:
word, count = getMaxWord(gameState)
print(count)
gameState = gameState.step(word)
print(gameState)

1352
Round 1:
FUZZY
XXXXX


In [180]:
gameState = WordleHardMode()
gameState.setTarget("APPLE")
while True:
    word, count = getMaxWord(gameState)
    gameState = gameState.step(word)
    print(gameState)
    if len(gameState.validWords) == 1:
        gameState = gameState.step(gameState.validWords[0])
        print(gameState)
        print("Finished")
        break

Round 1:
FUZZY
XXXXX
Round 2:
VIVID
XXXXX
Round 3:
BONGO
XXXXX
Round 4:
MAMMA
XYXXY
Round 5:
WREAK
XXYYX
Round 6:
ASSET
GXXYX
Round 7:
APPLE
GGGGG
Finished


In [157]:
gameState = WordleHardMode()
gameState.setTarget("APPLE")

In [159]:
for guess in ["FUZZY", "VIVID", "BONGO", "MAMMA", "WREAK"]:
    gameState = gameState.step(guess)

In [161]:
print(gameState)

Round 5:
WREAK
XXYYX


In [163]:
gs = gameState.step("ASSET")

In [164]:
gs.validWords

['APPLE']